# Paul the Octopus 2022
Elton Saheki esaheki@ciandt.com
***

# Data Analysis

In [1]:
import pandas as pd
import numpy as np

import google

from google.cloud import aiplatform
from google.cloud import bigquery

## Read data from BigQuery

In [2]:
credentials, project_id = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient = bigquery.Client(credentials=credentials, project=project_id)

In [3]:
query = "SELECT * FROM phoenix-cit.paul_2022.historical_results"
historical_raw = bqclient.query(query).result().to_dataframe();

In [4]:
query = "SELECT * FROM phoenix-cit.paul_2022.ranking"
ranking_raw = bqclient.query(query).result().to_dataframe();

## Data cleanup and exploring

In [5]:
historical_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44060 entries, 0 to 44059
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        44060 non-null  object 
 1   home_team   44060 non-null  object 
 2   away_team   44060 non-null  object 
 3   home_score  44059 non-null  float64
 4   away_score  44059 non-null  float64
 5   tournament  44060 non-null  object 
 6   city        44060 non-null  object 
 7   country     44060 non-null  object 
 8   neutral     44060 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.7+ MB


In [6]:
ranking_raw = ranking_raw[ranking_raw["rank_date"] != "rank_date"]

In [7]:
ranking_raw["rank_date"] = pd.to_datetime(ranking_raw["rank_date"], format='%Y-%m-%d')
ranking_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63916 entries, 0 to 63916
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   rank             63916 non-null  object        
 1   country_full     63916 non-null  object        
 2   country_abrv     63916 non-null  object        
 3   total_points     63916 non-null  object        
 4   previous_points  63916 non-null  object        
 5   rank_change      63916 non-null  object        
 6   confederation    63916 non-null  object        
 7   rank_date        63916 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 4.4+ MB


In [8]:
historical_raw["date"] = pd.to_datetime(historical_raw["date"])
historical_raw = historical_raw.replace({'home_team' : {
    'United States' : 'USA'
  }}
)
historical_raw = historical_raw.replace({'away_team' : {
    'United States' : 'USA'
  }}
)
historical_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44060 entries, 0 to 44059
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        44060 non-null  datetime64[ns]
 1   home_team   44060 non-null  object        
 2   away_team   44060 non-null  object        
 3   home_score  44059 non-null  float64       
 4   away_score  44059 non-null  float64       
 5   tournament  44060 non-null  object        
 6   city        44060 non-null  object        
 7   country     44060 non-null  object        
 8   neutral     44060 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), object(5)
memory usage: 2.7+ MB


## Find unmatched country names between rank and historical data

In [9]:
hist_unique_teams = pd.DataFrame(pd.unique((historical_raw['home_team']).append(historical_raw['away_team'])), columns=['Team'])
hist_unique_teams

,Team
0,Palestine
1,Cambodia
2,China PR
3,North Vietnam
4,North Korea
...,...
311,Crimea
312,Asturias
313,Surrey
314,Two Sicilies


In [10]:
rank_unique_teams = pd.DataFrame(pd.unique(ranking_raw['country_full']), columns=['country_full'])
rank_unique_teams

,country_full
0,Afghanistan
1,Anguilla
2,Albania
3,Algeria
4,Andorra
...,...
219,Serbia and Montenegro
220,São Tomé and Príncipe
221,Czechoslovakia
222,Türkiye


In [11]:
diff = pd.merge(hist_unique_teams, rank_unique_teams, left_on='Team', right_on='country_full', how='outer').query('Team != country_full')
diff.to_csv('store/diff.csv')
diff

,Team,country_full
3,North Vietnam,NaN
4,North Korea,NaN
5,Găgăuzia,NaN
6,Kyrgyzstan,NaN
7,Greenland,NaN
...,...,...
334,NaN,Zaire
335,NaN,Cabo Verde
336,NaN,Serbia and Montenegro
337,NaN,Türkiye


## Fixing country names

In [12]:
ranking_raw = ranking_raw.replace({'country_full' : {
    'Korea Republic' : 'South Korea',
    'Cabo Verde' : 'Cape Verde',
    'Cape Verde Islands' : 'Cape Verde',
    'Congo DR' : 'DR Congo',
    "Côte d'Ivoire" : 'Ivory Coast',
    'Curacao' : 'Curaçao',
    'FYR Macedonia' : 'North Macedonia',
    'IR Iran' : 'Iran',
    'Korea DPR' : 'North Korea',
    'Kyrgyz Republic' : 'Kyrgyzstan',
    'Sao Tome e Principe' : 'São Tomé and Príncipe	',
    'São Tomé e Príncipe' : 'São Tomé and Príncipe	',
    'St. Kitts and Nevis' : 'Saint Kitts and Nevis',
    'St. Lucia' : 'Saint Lucia',
    'St. Vincent / Grenadines' : 'Saint Vincent and the Grenadines',
    'St. Vincent and the Grenadines' : 'Saint Vincent and the Grenadines',
    'Swaziland' : 'Eswatini',
    'Türkiye' : 'Turkey',
    'US Virgin Islands' : 'United States Virgin Islands',
    'Zaire' : 'DR Congo'
  }}
)
ranking_raw

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,173,Afghanistan,AFG,48,102,-15,AFC,2006-07-12
1,190,Afghanistan,AFG,18,43,11,AFC,2007-04-18
2,182,Afghanistan,AFG,53,11,-14,AFC,2008-06-04
3,174,Afghanistan,AFG,61,10,-21,AFC,2011-04-13
4,179,Afghanistan,AFG,73,76,13,AFC,2011-07-27
...,...,...,...,...,...,...,...,...
63912,169,Saint Vincent and the Grenadines,VIN,117,88,-10,CONCACAF,2017-08-10
63913,192,United States Virgin Islands,VIR,17,0,-10,CONCACAF,2008-04-09
63914,78,Zambia,ZAM,418,365,-10,CAF,2014-10-23
63915,41,Zimbabwe,ZIM,38,35,-10,CAF,1995-02-20


## Merging Ranking data with Historical Results

In [13]:
historical_raw = historical_raw.sort_values(by="date")
ranking_raw = ranking_raw.sort_values(by="rank_date")

In [14]:
historical_home_rank = pd.merge_asof(historical_raw,ranking_raw, left_by='home_team', right_by='country_full',left_on='date', right_on='rank_date', direction='backward', allow_exact_matches=True)
historical_home_rank['home_rank'] = historical_home_rank['rank']
historical_home_rank['home_total_points'] = historical_home_rank['total_points']
historical_home_rank.drop(['rank','total_points','country_full','country_abrv','previous_points','rank_change','confederation','rank_date'], axis=1, inplace=True)
historical_home_rank

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_rank,home_total_points
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,NaN,NaN
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False,NaN,NaN
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,NaN,NaN
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,NaN,NaN
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
44055,2022-09-27,Malta,Israel,2.0,1.0,Friendly,Ta' Qali,Malta,False,169,974.68
44056,2022-09-27,Egypt,Liberia,3.0,0.0,Friendly,Alexandria,Egypt,False,40,1482.63
44057,2022-09-27,Ghana,Nicaragua,1.0,0.0,Friendly,Lorca,Spain,True,60,1393.47
44058,2022-09-27,Republic of Ireland,Armenia,3.0,2.0,UEFA Nations League,Dublin,Republic of Ireland,False,47,1442.48


In [15]:
historical_rank = pd.merge_asof(historical_home_rank,ranking_raw, left_by='away_team', right_by='country_full',left_on='date', right_on='rank_date', direction='backward', allow_exact_matches=True)
historical_rank['away_rank'] = historical_rank['rank']
historical_rank['away_total_points'] = historical_rank['total_points']
historical_rank.drop(['rank','total_points','country_full','country_abrv','previous_points','rank_change','confederation','rank_date'], axis=1, inplace=True)
historical_rank

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_rank,home_total_points,away_rank,away_total_points
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,NaN,NaN,NaN,NaN
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False,NaN,NaN,NaN,NaN
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,NaN,NaN,NaN,NaN
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,NaN,NaN,NaN,NaN
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44055,2022-09-27,Malta,Israel,2.0,1.0,Friendly,Ta' Qali,Malta,False,169,974.68,76,1316.35
44056,2022-09-27,Egypt,Liberia,3.0,0.0,Friendly,Alexandria,Egypt,False,40,1482.63,150,1051.25
44057,2022-09-27,Ghana,Nicaragua,1.0,0.0,Friendly,Lorca,Spain,True,60,1393.47,139,1079.18
44058,2022-09-27,Republic of Ireland,Armenia,3.0,2.0,UEFA Nations League,Dublin,Republic of Ireland,False,47,1442.48,92,1242.42


In [16]:
historical_rank = historical_rank[historical_rank['home_score'].notna()]
historical_rank = historical_rank[historical_rank['away_score'].notna()]

# ELO rating

Using ELO to standardize variations in FIFA Ranking and points algorithm

In [17]:
continental_championships = ['AFC Asian Cup','African Cup of Nations','Copa América','CONCACAF Championship','Oceania Nations Cup','UEFA Euro']
qualification_championships = ['FIFA World Cup qualification','AFC Asian Cup qualification','African Nations Championship qualification','Copa América qualification','CONCACAF Championship qualification','Oceania Nations Cup qualification','UEFA Euro qualification']

#tournament weight
def k_factor(tournament):
    k = 30
    if tournament == 'Friendly':
        k = 20
    elif tournament in qualification_championships:
        k = 25
    elif tournament in continental_championships:
        k = 50
    elif tournament == 'FIFA World Cup':
        k = 60
    return k

#goal difference weight
def g_factor(goals_a, goals_b):
    g = 1
    n = abs(goals_a - goals_b)
    if n == 2:
        g = 1.5
    elif n > 2:
        g = (11 + n) / 8
    return g

def expected_result(rating_a, rating_b, neutral):
    dr = rating_a - rating_b
    if not neutral:
      dr = (rating_a + 100) - rating_b
    we = (1 / (10**(-dr / 400) + 1))
    return [np.round(we, 3), 1-np.round(we, 3)]

def actual_result(goals_a, goals_b):
    if goals_a < goals_b:
        sa = 0
        sb = 1
    elif goals_a > goals_b:
        sa = 1
        sb = 0
    elif goals_a == goals_b:
        sa = 0.5
        sb = 0.5
    return [sa, sb]

def calculate_elo_rating(rating_a, rating_b, goals_a, goals_b, tournament, neutral):
    
    k = k_factor(tournament)
    g = g_factor(goals_a, goals_b)
    wa, wb = actual_result(goals_a, goals_b)
    wea, web = expected_result(rating_a, rating_b, neutral)

    new_rating_a = rating_a + k*g*(wa - wea)
    new_rating_b = rating_b + k*g*(wb - web)

    return new_rating_a.round(0), new_rating_b.round(0)

## Calculating ELO Ratings

In [18]:
current_elo = {}
for idx, row in historical_rank.iterrows():
    
    a_team = row['home_team']
    b_team = row['away_team']
    
    if a_team not in current_elo.keys():
        current_elo[a_team] = 2000
    
    if b_team not in current_elo.keys():
        current_elo[b_team] = 2000
    
    new_elo_a, new_elo_b = calculate_elo_rating(current_elo[a_team], current_elo[b_team], row['home_score'], row['away_score'], row['tournament'], row['neutral'])

    historical_rank.loc[idx,'k_factor'] = k_factor(row['tournament'])
    historical_rank.loc[idx,'home_elo_after'] = new_elo_a
    historical_rank.loc[idx,'away_elo_after'] = new_elo_b 
    historical_rank.loc[idx,'home_elo_before'] = current_elo[a_team]
    historical_rank.loc[idx,'away_elo_before'] = current_elo[b_team]

    current_elo[a_team] = new_elo_a
    current_elo[b_team] = new_elo_b

In [19]:
latest_elo = pd.DataFrame.from_dict(current_elo, orient='index', columns=['Elo'])
latest_elo.to_csv('store/latest_elo.csv')

In [20]:
historical_rank['score_diff'] = (historical_rank['home_score'] - historical_rank['away_score'])
historical_rank.to_csv('store/historical_rank.csv', index=False)

## Creating dummies for country values

In [21]:
home_team_dummies = pd.get_dummies(historical_rank['home_team'], prefix='dummy_home')
away_team_dummies = pd.get_dummies(historical_rank['away_team'], prefix='dummy_away')
historical_rank_dummies = historical_rank.join(home_team_dummies)
historical_rank_dummies = historical_rank_dummies.join(away_team_dummies)
historical_rank_dummies

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_rank,...,dummy_away_Western Sahara,dummy_away_Yemen,dummy_away_Yemen DPR,dummy_away_Ynys Môn,dummy_away_Yorkshire,dummy_away_Yugoslavia,dummy_away_Zambia,dummy_away_Zanzibar,dummy_away_Zimbabwe,dummy_away_Åland Islands
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,NaN,...,0,0,0,0,0,0,0,0,0,0
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44054,2022-09-27,United Arab Emirates,Venezuela,0.0,4.0,Friendly,Wiener Neustadt,Austria,True,69,...,0,0,0,0,0,0,0,0,0,0
44055,2022-09-27,Malta,Israel,2.0,1.0,Friendly,Ta' Qali,Malta,False,169,...,0,0,0,0,0,0,0,0,0,0
44056,2022-09-27,Egypt,Liberia,3.0,0.0,Friendly,Alexandria,Egypt,False,40,...,0,0,0,0,0,0,0,0,0,0
44057,2022-09-27,Ghana,Nicaragua,1.0,0.0,Friendly,Lorca,Spain,True,60,...,0,0,0,0,0,0,0,0,0,0


In [22]:
historical_rank_dummies.to_csv('store/historical_rank_elo_dummies.csv', index=False)

In [23]:
historical_rank_elo_dummies = historical_rank_dummies

# Training ML Model

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
dummy_columns = [k for k in historical_rank_elo_dummies.columns if (k[:6]=='dummy_')]
predictors = ["k_factor","home_elo_before","away_elo_before","home_rank","away_rank","home_total_points","away_total_points","neutral"] + dummy_columns
len(predictors)

625

In [26]:
historical_rank_elo_dummies = historical_rank_elo_dummies.dropna()

In [27]:
target = historical_rank_elo_dummies[["home_score","away_score"]]
features = historical_rank_elo_dummies[predictors]

In [28]:
X_train,X_test,y_train,y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [29]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

params = {
    "n_estimators": 200,
    "max_depth": 6,
    "min_samples_split": 5,
    "learning_rate": 0.1,
    "random_state": 512,
    "loss": "squared_error",
}

gbr = MultiOutputRegressor(GradientBoostingRegressor(**params));
gbr.fit(X_train, y_train)

score = gbr.score(X_train, y_train)
print("Training score:", score)

score = gbr.score(X_test, y_test)
print("Test score:", score)

predictions = gbr.predict(X_test)

# Just ELO + Full history
# Training score: 0.6105493749837039
# Test score: 0.1995738325346768

# ELO + Rank + Points + Partial history rank doesn't go far into the past
# Training score: 0.4972803263176074
# Test score: 0.27695701519816174

# Training score: 0.5040949343969829
# Test score: 0.26064179164703016

# Only World Cup countries data
# Training score: 0.5833660931368544
# Test score: 0.16928229496056335

Training score: 0.5040949343969829
Test score: 0.26064179164703016


## Store trained model

In [30]:
from datetime import datetime
from joblib import dump, load

now = datetime.now()
date_time = now.strftime("%Y%m%d_%H%M%S")
filename = 'trained-model-' + date_time + '.joblib'

dump(gbr, filename) 

['trained-model-20221214_215201.joblib']

In [31]:
predicts = pd.DataFrame(predictions, columns=["pred_home_score","pred_away_score"])
test = X_test.join(y_test).reset_index(drop=True)
test[predicts.columns] = predicts

## View prediction results

In [32]:
test[['home_elo_before','away_elo_before','home_rank','away_rank','home_total_points','away_total_points','neutral','home_score','away_score','pred_home_score','pred_away_score']].round({'pred_home_score': 0, 'pred_away_score': 0})

,home_elo_before,away_elo_before,home_rank,away_rank,home_total_points,away_total_points,neutral,home_score,away_score,pred_home_score,pred_away_score
0,2460.0,2315.0,5,19,1161,903,False,2.0,2.0,2.0,1.0
1,2193.0,2048.0,48,83,677,399,False,1.0,2.0,2.0,1.0
2,2144.0,1877.0,43,172,40,1,False,2.0,0.0,3.0,0.0
3,1890.0,1868.0,147,143,185,189,True,1.0,1.0,1.0,1.0
4,2298.0,2421.0,30,10,757,1030,False,1.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4862,1909.0,2164.0,129,62,225,520,False,2.0,0.0,1.0,1.0
4863,1938.0,2024.0,106,76,1205,1319,False,2.0,1.0,1.0,1.0
4864,1993.0,2013.0,113,120,416,393,False,0.0,2.0,1.0,1.0
4865,2267.0,2188.0,42,57,632,542,False,2.0,2.0,2.0,1.0


# Run prediction

## Prepare matches dataset to run prediction

In [33]:
matches = pd.read_csv("store/matches-schedule-finals.csv", sep=",")

In [34]:
matches

,match,date,country1,country2,phase
0,1,3/12/2022,Netherlands,USA,16
1,2,3/12/2022,Argentina,Australia,16
2,3,5/12/2022,Japan,Croatia,16
3,4,4/12/2022,England,Senegal,16
4,5,4/12/2022,France,Poland,16
5,6,6/12/2022,Morocco,Spain,16
6,7,5/12/2022,Brazil,South Korea,16
7,8,6/12/2022,Portugal,Switzerland,16
8,9,9/12/2022,Netherlands,Argentina,8
9,10,9/12/2022,Croatia,Brazil,8


In [35]:
group_matches = matches[matches['phase'] == 2]
group_matches = group_matches[['match','date','country1','country2']]
group_matches = group_matches.rename(columns={"country1": "home_team", "country2": "away_team"})
group_matches['date'] = pd.to_datetime(group_matches["date"])
group_matches = group_matches.sort_values(by='date')
group_matches

,match,date,home_team,away_team
14,15,2022-12-17,Croatia,Morocco
15,16,2022-12-18,Argentina,France


In [36]:
group_matches_home_rank = pd.merge_asof(group_matches,ranking_raw, left_by='home_team', right_by='country_full',left_on='date', right_on='rank_date', direction='backward', allow_exact_matches=True)
group_matches_home_rank['home_rank'] = group_matches_home_rank['rank']
group_matches_home_rank['home_total_points'] = group_matches_home_rank['total_points']
group_matches_home_rank.drop(['rank','total_points','country_full','country_abrv','previous_points','rank_change','confederation','rank_date'], axis=1, inplace=True)

In [37]:
group_matches_rank = pd.merge_asof(group_matches_home_rank,ranking_raw, left_by='away_team', right_by='country_full',left_on='date', right_on='rank_date', direction='backward', allow_exact_matches=True)
group_matches_rank['away_rank'] = group_matches_rank['rank']
group_matches_rank['away_total_points'] = group_matches_rank['total_points']
group_matches_rank.drop(['rank','total_points','country_full','country_abrv','previous_points','rank_change','confederation','rank_date'], axis=1, inplace=True)

In [38]:
for idx, row in group_matches_rank.iterrows():
    a_team = row['home_team']
    b_team = row['away_team']
    
    group_matches_rank.loc[idx,'k_factor'] = 60
    group_matches_rank.loc[idx,'home_elo_before'] = current_elo[a_team]
    group_matches_rank.loc[idx,'away_elo_before'] = current_elo[b_team]
    group_matches_rank.loc[idx,'neutral'] = (a_team != 'Qatar') & (b_team != 'Qatar')

In [39]:
group_matches_rank

,match,date,home_team,away_team,home_rank,home_total_points,away_rank,away_total_points,k_factor,home_elo_before,away_elo_before,neutral
0,15,2022-12-17,Croatia,Morocco,12,1645.64,22,1563.5,60.0,2427.0,2331.0,True
1,16,2022-12-18,Argentina,France,3,1773.88,4,1759.78,60.0,2650.0,2537.0,True


## Create dummies for country values

In [40]:
home_team_dummies = pd.get_dummies(group_matches_rank['home_team'], prefix='dummy_home')
away_team_dummies = pd.get_dummies(group_matches_rank['away_team'], prefix='dummy_away')
group_matches_dummies = group_matches_rank.join(home_team_dummies)
group_matches_dummies = group_matches_dummies.join(away_team_dummies)
group_matches_dummies

,match,date,home_team,away_team,home_rank,home_total_points,away_rank,away_total_points,k_factor,home_elo_before,away_elo_before,neutral,dummy_home_Argentina,dummy_home_Croatia,dummy_away_France,dummy_away_Morocco
0,15,2022-12-17,Croatia,Morocco,12,1645.64,22,1563.5,60.0,2427.0,2331.0,True,0,1,0,1
1,16,2022-12-18,Argentina,France,3,1773.88,4,1759.78,60.0,2650.0,2537.0,True,1,0,1,0


In [41]:
col_list = historical_rank_elo_dummies.columns.difference(group_matches_dummies.columns).tolist()
for col in col_list:
    group_matches_dummies[col] = 0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
dummy_columns = [k for k in group_matches_dummies.columns if (k[:6]=='dummy_')]
predictors_b = ["k_factor","home_elo_before","away_elo_before","home_rank","away_rank","home_total_points","away_total_points","neutral"] + dummy_columns
len(predictors_b)

625

In [43]:
group_matches_dummies[predictors_b]

,k_factor,home_elo_before,away_elo_before,home_rank,away_rank,home_total_points,away_total_points,neutral,dummy_home_Argentina,dummy_home_Croatia,...,dummy_home_Yemen,dummy_home_Yemen DPR,dummy_home_Ynys Môn,dummy_home_Yorkshire,dummy_home_Yoruba Nation,dummy_home_Yugoslavia,dummy_home_Zambia,dummy_home_Zanzibar,dummy_home_Zimbabwe,dummy_home_Åland Islands
0,60.0,2427.0,2331.0,12,22,1645.64,1563.5,True,0,1,...,0,0,0,0,0,0,0,0,0,0
1,60.0,2650.0,2537.0,3,4,1773.88,1759.78,True,1,0,...,0,0,0,0,0,0,0,0,0,0


## Run prediction

In [44]:
gbr = load('trained-model-20221117_010505.joblib') 
predictions = gbr.predict(group_matches_dummies[predictors_b])

score_predictions = pd.DataFrame(predictions, columns=["pred_home_score","pred_away_score"])
results = group_matches_dummies.reset_index(drop=True)
results[predicts.columns] = score_predictions

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [45]:
results = results[['match','date','home_team','away_team','home_elo_before','away_elo_before','home_rank','away_rank','home_total_points','away_total_points','neutral','pred_home_score','pred_away_score']]
results = results.rename(columns = {'home_team':'home', 'away_team':'away', 'pred_home_score':'home_score', 'pred_away_score':'away_score'})
results = results.sort_values(by='match').reset_index(drop=True)

In [46]:
results

,match,date,home,away,home_elo_before,away_elo_before,home_rank,away_rank,home_total_points,away_total_points,neutral,home_score,away_score
0,15,2022-12-17,Croatia,Morocco,2427.0,2331.0,12,22,1645.64,1563.5,True,1.521507,0.884454
1,16,2022-12-18,Argentina,France,2650.0,2537.0,3,4,1773.88,1759.78,True,0.759512,1.142900


## Export predictions

In [47]:
export = results[['home','home_score','away_score','away']].round({'home_score': 0, 'away_score': 0})
export = export.astype({'home_score': 'int32','away_score': 'int32' })
export

,home,home_score,away_score,away
0,Croatia,2,1,Morocco
1,Argentina,1,1,France


In [48]:
export.to_csv('submit-finals.csv', index=False)